In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/25 12:35:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [5]:
df_green.rdd.take(5)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 12, 29, 32), lpep_dropoff_datetime=datetime.datetime(2020, 1, 20, 12, 40, 46), store_and_fwd_flag='N', RatecodeID=1, PULocationID=65, DOLocationID=189, passenger_count=1, trip_distance=2.06, fare_amount=10.0, extra=0.0, mta_tax=0.5, tip_amount=2.16, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=12.96, payment_type=1, trip_type=1.0, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 30, 12, 41), lpep_dropoff_datetime=datetime.datetime(2020, 1, 30, 13, 30), store_and_fwd_flag=None, RatecodeID=None, PULocationID=117, DOLocationID=188, passenger_count=None, trip_distance=13.11, fare_amount=38.57, extra=2.75, mta_tax=0.0, tip_amount=0.0, tolls_amount=2.29, ehail_fee=None, improvement_surcharge=0.3, total_amount=43.91, payment_type=None, trip_type=None, congestion_surcharge=None),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 16, 22, 30,

In [7]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [10]:
from datetime import datetime

In [13]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [15]:
rows = rdd.take(10)
row = rows[0]

datetime.datetime(2020, 1, 20, 12, 0)

In [20]:
def prepare_for_grouping(row):
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    
    value = (amount, count)
    
    return (key, value)

In [24]:
def reduce(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
        
    return (output_amount, output_count)
    

In [32]:
from collections import namedtuple

In [33]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [34]:
def unwrap(row): 
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1], 
        revenue=row[1][0], 
        count=row[1][1])

In [38]:
from pyspark.sql import types

In [40]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True), 
    types.StructField('zone', types.IntegerType(), True), 
    types.StructField('revenue', types.DoubleType(), True), 
    types.StructField('count', types.IntegerType(), True)
])

In [41]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(reduce) \
    .map(unwrap) \
    .toDF(result_schema)

In [43]:
df_result.write.parquet('tmp/green_revenue_rdd')

In [5]:
columns = [
    'vendorID',
    'lpep_pickup_datetime',
    'PULocationID',
    'DOLocationID',
    'trip_distance',
]

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [13]:
import pandas as pd

In [15]:
rows = duration_rdd.take(10)

In [20]:
# model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [21]:
def apply_model_in_batch(rows): 
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions
    
    for row in df.itertuples():
          yield row

In [23]:
df_predictions = duration_rdd \
    .mapPartitions(apply_model_in_batch) \
    .toDF() \
    .drop('Index')

In [24]:
df_predictions.show()

+--------+--------------------+------------+------------+-------------+------------------+
|vendorID|lpep_pickup_datetime|PULocationID|DOLocationID|trip_distance|predicted_duration|
+--------+--------------------+------------+------------+-------------+------------------+
|       2|                  {}|          65|         189|         2.06|              10.3|
|       2|                  {}|         117|         188|        13.11|             65.55|
|       2|                  {}|          17|         217|         0.47|2.3499999999999996|
|       2|                  {}|          75|          74|         0.56|2.8000000000000003|
|       2|                  {}|          41|          74|         0.69|3.4499999999999997|
|       2|                  {}|          43|         140|         2.05|             10.25|
|       2|                  {}|          92|          92|         1.06| 5.300000000000001|
|       2|                  {}|         181|         177|         4.63|             23.15|